In [21]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"  # You can also use fine-tuned versions if available
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [35]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredURLLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline


In [23]:


loaders = UnstructuredURLLoader(urls=[
    "https://www.indianhealthyrecipes.com/chicken-kebab-recipe-chicken-kabab/",
    "https://cooktoria.com/easy-chicken-kebabs/"
])
data = loaders.load() 


In [26]:
# Use Hugging Face embeddings
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

In [27]:

# Create a FAISS index
faiss_index = FAISS.from_documents(data, embeddings)

In [47]:
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    temperature=0.7,
    top_k=50
)
llm = HuggingFacePipeline(pipeline=generation_pipeline)

Device set to use mps:0


In [49]:
llm = HuggingFacePipeline(pipeline=generation_pipeline)

In [55]:
# Create Retrieval-Augmented Generation chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_index.as_retriever(),
    chain_type="refine"
)

In [ ]:
query = "How to make a kebab"
result = qa_chain.run(query)
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
